In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Input, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# Load datasets
L128 = pd.read_csv('128_4_2_dataset.csv')
L256 = pd.read_csv('256_4_2_dataset.csv')
L512 = pd.read_csv('512_4_2_dataset.csv')
L1024 = pd.read_csv('1024_4_2_dataset.csv')
K2 = pd.read_csv('512_2_2_dataset.csv')
K8 = pd.read_csv('512_8_2_dataset.csv')
Q3 = pd.read_csv('512_4_3_dataset.csv')
Q4 = pd.read_csv('512_4_4_dataset.csv')
L128_K1 = pd.read_csv('128_1_2_dataset.csv')
L256_K2 = pd.read_csv('256_2_2_dataset.csv')
L1024_K8 = pd.read_csv('1024_8_2_dataset.csv')
print("Data loaded successfully.")

Data loaded successfully.


In [3]:
# This function is used to create a CNN Model with variable input shape.
# Different input shapes are needed depending on Bloom filter size l
def create_model(input_shape):
    model = Sequential()

    # Input layer
    model.add(Input(shape=input_shape))

    # Convolutional layers
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(MaxPooling1D())
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(MaxPooling1D())
    model.add(Dropout(0.2))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'precision', 'recall'])

    return model

# Pre-create models for each dataset size
model_128 = create_model(input_shape=(128, 1))
model_256 = create_model(input_shape=(256, 1))
model_512 = create_model(input_shape=(512, 1))
model_1024 = create_model(input_shape=(1024, 1))
model_2048 = create_model(input_shape=(2048, 1))
print("Models created successfully.")

Models created successfully.


In [4]:
# These two functions are used to remove redundancies (No need to explicitly split, train, and evaluate for each dataset)

# Function to convert binary strings to integers and split dataset into training, validation, testing.
def convert_and_split (data, labels, test_size=0.3, val_size=1/3, random_state=42):
    # Map binary strings to integers
    X = np.array([list(map(int, x)) for x in data])
    y = labels

    # Split dataset
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=test_size, random_state=random_state)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=test_size, random_state=random_state)

    return X_train, y_train, X_test, y_test, X_val, y_val

# Function to train and evaluate model. 
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, X_val, y_val, epochs=8, batch_size=128):

    # Train
    history = model.fit(
        X_train, 
        y_train, 
        epochs=epochs, 
        batch_size=batch_size, 
        validation_data=(X_val, y_val)
    )

    # Evaluate
    loss, accuracy, precision, recall = model.evaluate(X_test, y_test)

    # Print results
    print(f"Accuracy: {round(accuracy, 4)}")
    print(f"Precision: {round(precision, 4)}")
    print(f"Recall: {round(recall, 4)}")
    print(f"Loss: {round(loss, 4)}")

    # Return history for graph plotting
    return history.history

In [5]:
# Note: some experiments were re-run after the final evaluation, so not all results are the exact same as on the thesis paper. 
# However, they are very similar, with small changes due to randomness

# l=128, k=4, q=2 dataset evaluation
X_128 = L128['encodings']
y_128 = L128['label']

# Split the dataset
X_train, y_train, X_test, y_test, X_val, y_val = convert_and_split(X_128, y_128)
print("Data split successfully")

# Train and evaluate the model
results_128 = train_and_evaluate_model(model_128, X_train, y_train, X_test, y_test, X_val, y_val)

Data split successfully
Epoch 1/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 35s 13ms/step - accuracy: 0.7833 - loss: 0.4486 - precision: 0.7520 - recall: 0.7614 - val_accuracy: 0.8253 - val_loss: 0.3824 - val_precision: 0.7936 - val_recall: 0.8202
Epoch 2/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 32s 13ms/step - accuracy: 0.8247 - loss: 0.3815 - precision: 0.7880 - recall: 0.8300 - val_accuracy: 0.8337 - val_loss: 0.3637 - val_precision: 0.8005 - val_recall: 0.8335
Epoch 3/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 32s 13ms/step - accuracy: 0.8344 - loss: 0.3634 - precision: 0.8022 - recall: 0.8339 - val_accuracy: 0.8383 - val_loss: 0.3560 - val_precision: 0.8148 - val_recall: 0.8233
Epoch 4/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 33s 13ms/step - accuracy: 0.8418 - loss: 0.3503 - precision: 0.8113 - recall: 0.8413 - val_accuracy: 0.8421 - val_loss: 0.3481 - val_precision: 0.8198 - val_recall: 0.8262
Epoch 5/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 32s 13ms/step - accuracy: 0.8458 - loss: 0.3411 - precision: 0.8168 - recall: 0.843

In [17]:
# l=256, k=4, q=2 dataset evaluation
X_256 = L256['encodings']
y_256 = L256['label']

# Split the dataset
X_train, y_train, X_test, y_test, X_val, y_val = convert_and_split(X_256, y_256)
print("Data split successfully")

# Train and evaluate the model
results_256 = train_and_evaluate_model(model_256, X_train, y_train, X_test, y_test, X_val, y_val)

Data split successfully
Epoch 1/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 66s 25ms/step - accuracy: 0.8423 - loss: 0.3369 - precision: 0.8256 - recall: 0.8115 - val_accuracy: 0.8856 - val_loss: 0.2606 - val_precision: 0.8349 - val_recall: 0.9257
Epoch 2/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 64s 25ms/step - accuracy: 0.8885 - loss: 0.2586 - precision: 0.8598 - recall: 0.8966 - val_accuracy: 0.8951 - val_loss: 0.2449 - val_precision: 0.8580 - val_recall: 0.9154
Epoch 3/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 64s 25ms/step - accuracy: 0.8942 - loss: 0.2461 - precision: 0.8670 - recall: 0.9014 - val_accuracy: 0.8994 - val_loss: 0.2354 - val_precision: 0.8616 - val_recall: 0.9217
Epoch 4/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 65s 25ms/step - accuracy: 0.8979 - loss: 0.2362 - precision: 0.8715 - recall: 0.9042 - val_accuracy: 0.9013 - val_loss: 0.2300 - val_precision: 0.8889 - val_recall: 0.8890
Epoch 5/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 65s 25ms/step - accuracy: 0.9023 - loss: 0.2275 - precision: 0.8778 - recall: 0.906

In [18]:
# l=512, k=4, q=2 dataset evaluation
X_512 = L512['encodings']
y_512 = L512['label']

# Split the dataset
X_train, y_train, X_test, y_test, X_val, y_val = convert_and_split(X_512, y_512)
print("Data split successfully")

# Train and evaluate the model
results_512 = train_and_evaluate_model(model_512, X_train, y_train, X_test, y_test, X_val, y_val)

Data split successfully
Epoch 1/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 142s 55ms/step - accuracy: 0.8842 - loss: 0.2572 - precision: 0.8662 - recall: 0.8739 - val_accuracy: 0.9262 - val_loss: 0.1760 - val_precision: 0.9135 - val_recall: 0.9211
Epoch 2/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 139s 54ms/step - accuracy: 0.9255 - loss: 0.1775 - precision: 0.9105 - recall: 0.9243 - val_accuracy: 0.9350 - val_loss: 0.1574 - val_precision: 0.9343 - val_recall: 0.9184
Epoch 3/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 138s 54ms/step - accuracy: 0.9364 - loss: 0.1508 - precision: 0.9258 - recall: 0.9315 - val_accuracy: 0.9371 - val_loss: 0.1516 - val_precision: 0.9369 - val_recall: 0.9204
Epoch 4/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 137s 54ms/step - accuracy: 0.9439 - loss: 0.1344 - precision: 0.9348 - recall: 0.9398 - val_accuracy: 0.9388 - val_loss: 0.1490 - val_precision: 0.9350 - val_recall: 0.9268
Epoch 5/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 137s 54ms/step - accuracy: 0.9492 - loss: 0.1218 - precision: 0.9402 - recall: 

In [19]:
# l=1024, k=4, q=2 dataset evaluation
X_1024 = L1024['encodings']
y_1024 = L1024['label']

# Split the dataset
X_train, y_train, X_test, y_test, X_val, y_val = convert_and_split(X_1024, y_1024)
print("Data split successfully")

# Train and evaluate the model
results_1024 = train_and_evaluate_model(model_1024, X_train, y_train, X_test, y_test, X_val, y_val)

Data split successfully
Epoch 1/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 284s 110ms/step - accuracy: 0.8993 - loss: 0.2222 - precision: 0.8877 - recall: 0.8842 - val_accuracy: 0.9363 - val_loss: 0.1553 - val_precision: 0.9347 - val_recall: 0.9211
Epoch 2/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 280s 109ms/step - accuracy: 0.9360 - loss: 0.1538 - precision: 0.9226 - recall: 0.9348 - val_accuracy: 0.9411 - val_loss: 0.1424 - val_precision: 0.9353 - val_recall: 0.9320
Epoch 3/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 280s 109ms/step - accuracy: 0.9419 - loss: 0.1395 - precision: 0.9300 - recall: 0.9406 - val_accuracy: 0.9425 - val_loss: 0.1387 - val_precision: 0.9140 - val_recall: 0.9610
Epoch 4/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 279s 109ms/step - accuracy: 0.9490 - loss: 0.1246 - precision: 0.9389 - recall: 0.9471 - val_accuracy: 0.9475 - val_loss: 0.1290 - val_precision: 0.9390 - val_recall: 0.9431
Epoch 5/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 279s 109ms/step - accuracy: 0.9530 - loss: 0.1140 - precision: 0.9442 - rec

In [20]:
# l=512, k=2, q=2 dataset evaluation
X_K2 = K2['encodings']
y_K2 = K2['label']

# Split the dataset
X_train, y_train, X_test, y_test, X_val, y_val = convert_and_split(X_K2, y_K2)
print("Data split successfully")

# Train and evaluate the model
results_K2 = train_and_evaluate_model(model_512, X_train, y_train, X_test, y_test, X_val, y_val)

Data split successfully
Epoch 1/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 144s 56ms/step - accuracy: 0.9207 - loss: 0.1887 - precision: 0.9048 - recall: 0.9191 - val_accuracy: 0.9395 - val_loss: 0.1483 - val_precision: 0.9327 - val_recall: 0.9310
Epoch 2/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 143s 56ms/step - accuracy: 0.9438 - loss: 0.1336 - precision: 0.9341 - recall: 0.9403 - val_accuracy: 0.9419 - val_loss: 0.1394 - val_precision: 0.9243 - val_recall: 0.9467
Epoch 3/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 143s 56ms/step - accuracy: 0.9510 - loss: 0.1180 - precision: 0.9424 - recall: 0.9480 - val_accuracy: 0.9436 - val_loss: 0.1399 - val_precision: 0.9284 - val_recall: 0.9462
Epoch 4/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 144s 56ms/step - accuracy: 0.9557 - loss: 0.1069 - precision: 0.9492 - recall: 0.9518 - val_accuracy: 0.9435 - val_loss: 0.1391 - val_precision: 0.9360 - val_recall: 0.9369
Epoch 5/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 145s 57ms/step - accuracy: 0.9591 - loss: 0.0988 - precision: 0.9534 - recall: 

In [21]:
# l=512, k=8, q=2 dataset evaluation
X_K8 = K8['encodings']
y_K8 = K8['label']

# Split the dataset
X_train, y_train, X_test, y_test, X_val, y_val = convert_and_split(X_K8, y_K8)
print("Data split successfully")

# Train and evaluate the model
results_K8 = train_and_evaluate_model(model_512, X_train, y_train, X_test, y_test, X_val, y_val)

Data split successfully
Epoch 1/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 146s 57ms/step - accuracy: 0.8722 - loss: 0.3039 - precision: 0.8459 - recall: 0.8724 - val_accuracy: 0.9174 - val_loss: 0.1934 - val_precision: 0.8949 - val_recall: 0.9226
Epoch 2/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 146s 57ms/step - accuracy: 0.9218 - loss: 0.1851 - precision: 0.9065 - recall: 0.9188 - val_accuracy: 0.9269 - val_loss: 0.1742 - val_precision: 0.9029 - val_recall: 0.9362
Epoch 3/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 147s 57ms/step - accuracy: 0.9324 - loss: 0.1616 - precision: 0.9191 - recall: 0.9305 - val_accuracy: 0.9299 - val_loss: 0.1672 - val_precision: 0.9093 - val_recall: 0.9356
Epoch 4/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 147s 58ms/step - accuracy: 0.9395 - loss: 0.1451 - precision: 0.9288 - recall: 0.9361 - val_accuracy: 0.9315 - val_loss: 0.1640 - val_precision: 0.9136 - val_recall: 0.9343
Epoch 5/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 149s 58ms/step - accuracy: 0.9452 - loss: 0.1324 - precision: 0.9358 - recall: 

In [22]:
# l=512, k=4, q=3 dataset evaluation
X_Q3 = Q3['encodings']
y_Q3 = Q3['label']

# Split the dataset
X_train, y_train, X_test, y_test, X_val, y_val = convert_and_split(X_Q3, y_Q3)
print("Data split successfully")

# Train and evaluate the model
results_Q3 = train_and_evaluate_model(model_512, X_train, y_train, X_test, y_test, X_val, y_val)

Data split successfully
Epoch 1/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 149s 58ms/step - accuracy: 0.8475 - loss: 0.3372 - precision: 0.8164 - recall: 0.8480 - val_accuracy: 0.8974 - val_loss: 0.2370 - val_precision: 0.8594 - val_recall: 0.9195
Epoch 2/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 149s 58ms/step - accuracy: 0.8964 - loss: 0.2404 - precision: 0.8732 - recall: 0.8981 - val_accuracy: 0.9054 - val_loss: 0.2204 - val_precision: 0.8856 - val_recall: 0.9040
Epoch 3/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 146s 57ms/step - accuracy: 0.9047 - loss: 0.2203 - precision: 0.8851 - recall: 0.9034 - val_accuracy: 0.9093 - val_loss: 0.2119 - val_precision: 0.8942 - val_recall: 0.9028
Epoch 4/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 146s 57ms/step - accuracy: 0.9116 - loss: 0.2056 - precision: 0.8949 - recall: 0.9087 - val_accuracy: 0.9124 - val_loss: 0.2050 - val_precision: 0.9050 - val_recall: 0.8971
Epoch 5/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 146s 57ms/step - accuracy: 0.9175 - loss: 0.1937 - precision: 0.9022 - recall: 

In [23]:
# l=512, k=4, q=4 dataset evaluation
X_Q4 = Q4['encodings']
y_Q4 = Q4['label']

# Split the dataset
X_train, y_train, X_test, y_test, X_val, y_val = convert_and_split(X_Q4, y_Q4)
print("Data split successfully")

# Train and evaluate the model
results_Q4 = train_and_evaluate_model(model_512, X_train, y_train, X_test, y_test, X_val, y_val)

Data split successfully
Epoch 1/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 149s 58ms/step - accuracy: 0.8169 - loss: 0.3834 - precision: 0.7865 - recall: 0.8050 - val_accuracy: 0.8831 - val_loss: 0.2656 - val_precision: 0.8548 - val_recall: 0.8878
Epoch 2/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 149s 58ms/step - accuracy: 0.8810 - loss: 0.2694 - precision: 0.8567 - recall: 0.8806 - val_accuracy: 0.8879 - val_loss: 0.2553 - val_precision: 0.8664 - val_recall: 0.8840
Epoch 3/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 149s 58ms/step - accuracy: 0.8882 - loss: 0.2544 - precision: 0.8660 - recall: 0.8871 - val_accuracy: 0.8925 - val_loss: 0.2456 - val_precision: 0.8779 - val_recall: 0.8808
Epoch 4/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 149s 58ms/step - accuracy: 0.8953 - loss: 0.2394 - precision: 0.8754 - recall: 0.8913 - val_accuracy: 0.8928 - val_loss: 0.2442 - val_precision: 0.8556 - val_recall: 0.9129
Epoch 5/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 149s 58ms/step - accuracy: 0.9007 - loss: 0.2264 - precision: 0.8829 - recall: 

In [26]:
# l=128, k=1, q=2 dataset evaluation
X_L128_K1 = L128_K1['encodings']
y_L128_K1 = L128_K1['label']

# Split the dataset
X_train, y_train, X_test, y_test, X_val, y_val = convert_and_split(X_L128_K1, y_L128_K1)
print("Data split successfully")

# Train and evaluate the model
results_L128_K1 = train_and_evaluate_model(model_128, X_train, y_train, X_test, y_test, X_val, y_val)

Data split successfully
Epoch 1/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 44s 17ms/step - accuracy: 0.8350 - loss: 0.3553 - precision: 0.8147 - recall: 0.8126 - val_accuracy: 0.8788 - val_loss: 0.2783 - val_precision: 0.8697 - val_recall: 0.8556
Epoch 2/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 43s 17ms/step - accuracy: 0.8776 - loss: 0.2779 - precision: 0.8563 - recall: 0.8716 - val_accuracy: 0.8877 - val_loss: 0.2574 - val_precision: 0.8670 - val_recall: 0.8829
Epoch 3/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 43s 17ms/step - accuracy: 0.8884 - loss: 0.2551 - precision: 0.8679 - recall: 0.8831 - val_accuracy: 0.8933 - val_loss: 0.2478 - val_precision: 0.8826 - val_recall: 0.8765
Epoch 4/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 42s 17ms/step - accuracy: 0.8956 - loss: 0.2425 - precision: 0.8775 - recall: 0.8903 - val_accuracy: 0.8963 - val_loss: 0.2409 - val_precision: 0.8723 - val_recall: 0.8984
Epoch 5/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 42s 17ms/step - accuracy: 0.9014 - loss: 0.2288 - precision: 0.8854 - recall: 0.895

In [28]:
# l=256, k=2, q=2 dataset evaluation
X_L256_K2 = L256_K2['encodings']
y_L256_K2 = L256_K2['label']

# Split the dataset
X_train, y_train, X_test, y_test, X_val, y_val = convert_and_split(X_L256_K2, y_L256_K2)
print("Data split successfully")

# Train and evaluate the model
results_L128_K2 = train_and_evaluate_model(model_256, X_train, y_train, X_test, y_test, X_val, y_val)

Data split successfully
Epoch 1/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 86s 34ms/step - accuracy: 0.8823 - loss: 0.2693 - precision: 0.8554 - recall: 0.8857 - val_accuracy: 0.9078 - val_loss: 0.2159 - val_precision: 0.8718 - val_recall: 0.9292
Epoch 2/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 87s 34ms/step - accuracy: 0.9082 - loss: 0.2154 - precision: 0.8851 - recall: 0.9126 - val_accuracy: 0.9138 - val_loss: 0.2034 - val_precision: 0.9017 - val_recall: 0.9047
Epoch 3/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 85s 33ms/step - accuracy: 0.9138 - loss: 0.2021 - precision: 0.8933 - recall: 0.9154 - val_accuracy: 0.9143 - val_loss: 0.2006 - val_precision: 0.8805 - val_recall: 0.9340
Epoch 4/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 86s 33ms/step - accuracy: 0.9182 - loss: 0.1939 - precision: 0.8990 - recall: 0.9203 - val_accuracy: 0.9151 - val_loss: 0.1997 - val_precision: 0.8781 - val_recall: 0.9395
Epoch 5/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 85s 33ms/step - accuracy: 0.9203 - loss: 0.1882 - precision: 0.9017 - recall: 0.921

In [29]:
# l=1024, k=8, q=2 dataset evaluation
X_L1024_K8 = L1024_K8['encodings']
y_L1024_K8 = L1024_K8['label']

# Split the dataset
X_train, y_train, X_test, y_test, X_val, y_val = convert_and_split(X_L1024_K8, y_L1024_K8)
print("Data split successfully")

# Train and evaluate the model
results_L1024_K8 = train_and_evaluate_model(model_1024, X_train, y_train, X_test, y_test, X_val, y_val)

Data split successfully
Epoch 1/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 310s 121ms/step - accuracy: 0.9239 - loss: 0.1829 - precision: 0.9087 - recall: 0.9220 - val_accuracy: 0.9441 - val_loss: 0.1351 - val_precision: 0.9287 - val_recall: 0.9470
Epoch 2/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 308s 120ms/step - accuracy: 0.9509 - loss: 0.1204 - precision: 0.9432 - recall: 0.9470 - val_accuracy: 0.9479 - val_loss: 0.1279 - val_precision: 0.9431 - val_recall: 0.9395
Epoch 3/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 304s 118ms/step - accuracy: 0.9596 - loss: 0.0997 - precision: 0.9533 - recall: 0.9561 - val_accuracy: 0.9486 - val_loss: 0.1267 - val_precision: 0.9414 - val_recall: 0.9429
Epoch 4/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 321s 125ms/step - accuracy: 0.9654 - loss: 0.0857 - precision: 0.9602 - recall: 0.9622 - val_accuracy: 0.9493 - val_loss: 0.1301 - val_precision: 0.9452 - val_recall: 0.9406
Epoch 5/8
2562/2562 ━━━━━━━━━━━━━━━━━━━━ 303s 118ms/step - accuracy: 0.9696 - loss: 0.0749 - precision: 0.9652 - rec